In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
# # Sample data (replace with your actual data)
# X_train = np.random.rand(100, 450).astype(np.float32)  # Sample input data
# y_train = np.random.rand(100, 60).astype(np.float32)   # Sample output labels

# # Convert data to PyTorch tensors
# X_train_tensor = torch.tensor(X_train)
# y_train_tensor = torch.tensor(y_train)

DATA_DIR = '../data'

with open(os.path.join(DATA_DIR, 'input_output_list.pickle'), 'rb') as f:
    a = pickle.load(f)

X = np.array([d[0] for d in a]).astype(np.float32)
y = np.array([d[1] for d in a]).astype(np.float32)

# Normalize input features
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

# Split data into train and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X_normalized, y, test_size=0.2, random_state=42)
X_train, X_temp, y_train, y_temp = train_test_split(X_normalized, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_val_tensor = torch.tensor(X_val)
y_val_tensor = torch.tensor(y_val)



In [3]:

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        # Experiment with hidden layer sizes:
        self.fc1 = nn.Linear(450, 256)  
        self.fc2 = nn.Linear(256, 128)  
        self.fc3 = nn.Linear(128, 64)  # Reduced layer sizes
        self.fc4 = nn.Linear(64, 60)

        self.dropout = nn.Dropout(0.2)  
        self.relu = nn.ReLU()  # ReLU for efficiency

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x) 
        x = self.relu(self.fc2(x))
        x = self.dropout(x) 
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


# Hyperparameter tuning loop
learning_rates = [0.001, 0.0007, 0.0005, 0.0003]

for lr in learning_rates:
    model = NeuralNetwork()  # New model instance for each learning rate
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training loop with early stopping
    best_val_loss = float('inf')
    patience = 5
    early_stopping_counter = 0

    for epoch in range(500):  # You can adjust the number of epochs
        # Forward pass
        model.train()
        outputs = model(X_train_tensor)
        train_loss = criterion(outputs, y_train_tensor)
    
        # Backward pass and optimization
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor)

        # Print progress
        print(f'Epoch [{epoch+1}/500], Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print("Early stopping...")
                break

    # Evaluate the model on test data
    model.eval()
    with torch.no_grad():
        test_outputs = model(torch.tensor(X_test))
        test_loss = criterion(test_outputs, torch.tensor(y_test))
        print(f'Test Loss: {test_loss.item():.4f}')






# Save the trained model
torch.save(model.state_dict(), 'further_improved_model.pth')

Epoch [1/500], Train Loss: 135362.0938, Val Loss: 135955.9531
Epoch [2/500], Train Loss: 135361.2188, Val Loss: 135955.1250
Epoch [3/500], Train Loss: 135360.4062, Val Loss: 135954.2812
Epoch [4/500], Train Loss: 135359.5781, Val Loss: 135953.4062
Epoch [5/500], Train Loss: 135358.7188, Val Loss: 135952.5000
Epoch [6/500], Train Loss: 135357.8125, Val Loss: 135951.5000
Epoch [7/500], Train Loss: 135356.7969, Val Loss: 135950.3906
Epoch [8/500], Train Loss: 135355.7031, Val Loss: 135949.1406
Epoch [9/500], Train Loss: 135354.4688, Val Loss: 135947.7344
Epoch [10/500], Train Loss: 135353.0312, Val Loss: 135946.0781
Epoch [11/500], Train Loss: 135351.3906, Val Loss: 135944.1406
Epoch [12/500], Train Loss: 135349.4219, Val Loss: 135941.8281
Epoch [13/500], Train Loss: 135347.1250, Val Loss: 135939.0781
Epoch [14/500], Train Loss: 135344.3750, Val Loss: 135935.7656
Epoch [15/500], Train Loss: 135341.1250, Val Loss: 135931.8594
Epoch [16/500], Train Loss: 135337.2969, Val Loss: 135927.2656
E